# Pre-processing and Data Cleaning

In [71]:
import pandas as pd
import numpy as np

year = 2025
games_df = pd.read_csv(f'game-data/{year}games.csv', index_col='Unnamed: 0')

In [72]:
games_df['Pre-Game Team Elo'] = ''
games_df['Pre-Game Opponent Elo'] = ''
games_df['Post-Game Team Elo'] = ''
games_df['Post-Game Opponent Elo'] = ''
games_df.tail(19)

,Date,Team,Conference,Opponent,Venue,Result,AdjO,AdjD,EffO,eFG%,...,Opponent Coach,Unknown,Game Importance,Team Points,Opponent Points,Point Differential,Pre-Game Team Elo,Pre-Game Opponent Elo,Post-Game Team Elo,Post-Game Opponent Elo
3279,2025-03-16,VCU,A10,George Mason,N,W,120.5,99.5,107.2,48.9,...,Tony Skinn,3.93792,0.766671,68,63,5,,,,
6737,2025-03-18,San Diego St.,MWC,North Carolina,N,L,103.4,120.2,96.9,46.6,...,Hubert Davis,-16.63610,0.894540,68,95,-27,,,,
4288,2025-03-18,Georgia Tech,ACC,Jacksonville St.,H,L,93.6,117.4,93.4,46.0,...,Ray Harper,-10.93530,0.544160,64,81,-17,,,,
4501,2025-03-18,St. Bonaventure,A10,Kent St.,H,L,90.1,103.2,80.4,41.0,...,Rob Senderoff,-1.61531,0.549910,56,75,-19,,,,
8204,2025-03-18,Arkansas St.,SB,Saint Louis,H,W,134.9,102.7,135.4,64.9,...,Josh Schertz,13.46730,0.537310,103,78,25,,,,
4289,2025-03-18,Jacksonville St.,CUSA,Georgia Tech,A,W,127.2,91.4,118.2,64.2,...,Damon Stoudamire,10.93530,0.734254,81,64,17,,,,
8205,2025-03-18,Saint Louis,A10,Arkansas St.,A,L,109.1,127.4,102.5,55.2,...,Brian Hodgson,-13.46730,0.768762,78,103,-25,,,,
1381,2025-03-18,Stanford,ACC,Cal St. Northridge,H,W,125.3,100.1,124.8,56.7,...,Andy Newman,11.35420,0.486997,87,70,17,,,,
6736,2025-03-18,North Carolina,ACC,San Diego St.,N,W,150.8,95.0,135.4,64.9,...,Brian Dutcher,16.63610,0.810532,95,68,27,,,,
1380,2025-03-18,Cal St. Northridge,BW,Stanford,A,L,106.0,116.9,100.4,49.2,...,Kyle Smith,-11.35420,0.817915,70,87,-17,,,,


In [73]:
print(games_df.columns)

Index(['Date', 'Team', 'Conference', 'Opponent', 'Venue', 'Result', 'AdjO',
       'AdjD', 'EffO', 'eFG%', 'TO%', 'Reb%', 'FTR', 'EffD', 'Opp eFG%',
       'Opp TO%', 'Opp Reb%', 'Opp FTR', 'G-SC', 'Opponent Conference',
       'Game Tempo', 'Game Unique ID', 'Coach', 'Opponent Coach', 'Unknown',
       'Game Importance', 'Team Points', 'Opponent Points',
       'Point Differential', 'Pre-Game Team Elo', 'Pre-Game Opponent Elo',
       'Post-Game Team Elo', 'Post-Game Opponent Elo'],
      dtype='object')


#### Consider new x_i's:
* (Date, Team, Conference, Opponent, Venue, Coach, Opponent Coach) -> will have to project everything else

# Elo Rating Adjustment with Point Differential

To better reflect the impact of **blowout wins**, we adjust Elo ratings by incorporating **Margin of Victory (MOV)**.

## **Adjusted Elo Formula with MOV**
$$
R_A' = R_A + K \cdot (W_A - P_A) \cdot f(\text{MOV})
$$

$$
R_B' = R_B + K \cdot (W_B - P_B) \cdot f(\text{MOV})
$$

Where:
- \( R_A', R_B' \) = Updated Elo ratings for Team A and Team B  
- \( R_A, R_B \) = Pre-game Elo ratings  
- \( K \) = Scaling factor (typically **20-40**)  
- \( W_A, W_B \) = Win values (**1 for a win, 0 for a loss**)  
- \( P_A, P_B \) = Expected probabilities, given by:  

$$
P_A = \frac{1}{1 + 10^{(R_B - R_A)/400}}
$$

- \( \text{MOV} \) = Margin of Victory (Point Differential)  
- \( f(\text{MOV}) \) = A function that increases Elo changes for larger wins:

$$
f(\text{MOV}) = \ln(\text{MOV} + 1) \cdot \frac{2.2}{(R_A - R_B) \times 0.001 + 2.2}
$$

## **Explanation**
- **Larger MOV values** increase the Elo adjustment.
- **Close games** (small MOV) result in **standard Elo changes**.
- **Blowouts** (high MOV) result in **bigger Elo adjustments**, but are limited to avoid extreme shifts.


In [74]:
import math

team_elos = {team: 1500 for team in games_df['Team'].unique()}
processed_games = {} # {'Game Unique ID': ((R_A, R_B, R_A', P_B'))}

def expected_win_probability(team_A, team_B):
    R_A = team_elos[team_A]
    R_B = team_elos[team_B]
    R_D = R_B - R_A # elo differential
    P_A = 1 / (1 + 10 ** (R_D / 400))
    return P_A

def f_mov(R_A, R_B, MOV):
    return math.log(MOV + 1) * 2.2 / ((R_A - R_B) * 0.001 + 2.2)

def calulate_elo(team_A, team_B, point_differential, game_unique_id, K = 20):
    R_A = team_elos[team_A]
    R_B = team_elos[team_B]

    P_A = expected_win_probability(team_A, team_B)
    P_B = 1 - P_A

    if point_differential > 0: # team A won    
        elo_change = int(K * (1 - P_A) * f_mov(R_A, R_B, point_differential))
        R_A_NEW = team_elos[team_A] + elo_change
        R_B_NEW = team_elos[team_B] - elo_change
    else: # team B won
        elo_change = int(K * (1 - P_B) * f_mov(R_B, R_A, -point_differential))
        R_A_NEW = team_elos[team_A] - elo_change
        R_B_NEW = team_elos[team_B] + elo_change

    processed_games[game_unique_id] = ((R_A, R_B, R_A_NEW, R_B_NEW))
    return (R_A_NEW, R_B_NEW)


In [75]:
pre_game_team_elo = []
pre_game_opponent_elo = []
post_game_team_elo = []
post_game_opponent_elo = []

for i, row in games_df.iterrows():
    team_A, team_B, point_diff, game_unique_id = row['Team'], row['Opponent'], row['Point Differential'], row['Game Unique ID']
    
    if game_unique_id in processed_games:
        R_B, R_A, R_B_NEW, R_A_NEW = processed_games[game_unique_id]
    else:
        R_A, R_B = team_elos[team_A], team_elos[team_B]
        R_A_NEW, R_B_NEW = calulate_elo(team_A, team_B, point_diff, game_unique_id)
        team_elos[team_A] = R_A_NEW
        team_elos[team_B] = R_B_NEW

    pre_game_team_elo.append(R_A)
    pre_game_opponent_elo.append(R_B)
    post_game_team_elo.append(R_A_NEW)
    post_game_opponent_elo.append(R_B_NEW)

games_df["Pre-Game Team Elo"] = pre_game_team_elo
games_df["Pre-Game Opponent Elo"] = pre_game_opponent_elo
games_df["Post-Game Team Elo"] = post_game_team_elo
games_df["Post-Game Opponent Elo"] = post_game_opponent_elo
team_elos_2025 = sorted(team_elos.items(), key=lambda x: x[1], reverse=True)

for i, (team, elo) in enumerate(team_elos_2025[:64]):
    print(i + 1, team, elo)

1 Houston 1932
2 Florida 1929
3 Duke 1918
4 St. John's 1887
5 Michigan St. 1856
6 Auburn 1839
7 Gonzaga 1829
8 VCU 1827
9 Tennessee 1827
10 Maryland 1819
11 UC San Diego 1817
12 Saint Mary's 1815
13 Clemson 1811
14 Alabama 1808
15 Louisville 1797
16 Drake 1794
17 Colorado St. 1789
18 Memphis 1778
19 Texas Tech 1773
20 Wisconsin 1763
21 BYU 1763
22 Arizona 1756
23 Michigan 1750
24 High Point 1740
25 Iowa St. 1733
26 New Mexico 1733
27 Liberty 1731
28 Connecticut 1731
29 North Carolina 1728
30 McNeese St. 1728
31 Creighton 1718
32 Illinois 1716
33 Oregon 1714
34 Yale 1713
35 George Mason 1711
36 Texas A&M 1709
37 Boise St. 1705
38 Akron 1702
39 Xavier 1701
40 Kentucky 1697
41 Grand Canyon 1695
42 UCLA 1694
43 Missouri 1694
44 Purdue 1693
45 UC Irvine 1691
46 Chattanooga 1688
47 Robert Morris 1681
48 Mississippi 1678
49 UNC Wilmington 1676
50 Marquette 1674
51 Georgia 1674
52 Santa Clara 1674
53 Utah St. 1671
54 Lipscomb 1669
55 Oklahoma 1668
56 Dayton 1667
57 Baylor 1664
58 San Francisco

In [76]:
games_df[['Date', 'Team', 'Opponent', 'Result', 'Point Differential', 'Pre-Game Team Elo',
       'Pre-Game Opponent Elo', 'Post-Game Team Elo', 'Post-Game Opponent Elo']].tail(15)

,Date,Team,Opponent,Result,Point Differential,Pre-Game Team Elo,Pre-Game Opponent Elo,Post-Game Team Elo,Post-Game Opponent Elo
8204,2025-03-18,Arkansas St.,Saint Louis,W,25,1634,1603,1663,1574
4289,2025-03-18,Jacksonville St.,Georgia Tech,W,17,1576,1564,1603,1537
8205,2025-03-18,Saint Louis,Arkansas St.,L,-25,1603,1634,1574,1663
1381,2025-03-18,Stanford,Cal St. Northridge,W,17,1593,1603,1622,1574
6736,2025-03-18,North Carolina,San Diego St.,W,27,1700,1658,1728,1630
1380,2025-03-18,Cal St. Northridge,Stanford,L,-17,1603,1593,1574,1622
11040,2025-03-18,Oklahoma St.,Wichita St.,W,10,1506,1530,1531,1505
1726,2025-03-18,Chattanooga,Middle Tennessee,W,6,1675,1583,1688,1570
169,2025-03-18,Saint Francis,Alabama St.,L,-2,1454,1465,1444,1475
168,2025-03-18,Alabama St.,Saint Francis,W,2,1465,1454,1475,1444


### First Four (Second Day 2025) Elos

In [77]:
print('Texas Elo:', team_elos['Texas'])
print('Xavier Elo:', team_elos['Xavier'] , '\n')

print("Mount St. Mary's Elo:", team_elos["Mount St. Mary's"])
print('American Elo:', team_elos['American'])



Texas Elo: 1602
Xavier Elo: 1701 

Mount St. Mary's Elo: 1550
American Elo: 1555


### Add Pre-Game and Post-Game Elos to Each csv

In [149]:
years = [year for year in range(2008, 2025 + 1)]

for j, year in enumerate(years):
    games_df = pd.read_csv(f'game-data/{year}games.csv', index_col='Unnamed: 0')
    games_df['Pre-Game Team Elo'] = ''
    games_df['Pre-Game Opponent Elo'] = ''
    games_df['Post-Game Team Elo'] = ''
    games_df['Post-Game Opponent Elo'] = ''

    team_elos = {team: 1500 for team in games_df['Team'].unique()}
    processed_games = {} # {'Game Unique ID': ((R_A, R_B, R_A', P_B'))}

    pre_game_team_elo = []
    pre_game_opponent_elo = []
    post_game_team_elo = []
    post_game_opponent_elo = []

    for i, row in games_df.iterrows():
        team_A, team_B, point_diff, game_unique_id = row['Team'], row['Opponent'], row['Point Differential'], row['Game Unique ID']
        
        if game_unique_id in processed_games:
            R_B, R_A, R_B_NEW, R_A_NEW = processed_games[game_unique_id]
        else:
            R_A, R_B = team_elos[team_A], team_elos[team_B]
            R_A_NEW, R_B_NEW = calulate_elo(team_A, team_B, point_diff, game_unique_id, K=20)
            team_elos[team_A] = R_A_NEW
            team_elos[team_B] = R_B_NEW

        pre_game_team_elo.append(R_A)
        pre_game_opponent_elo.append(R_B)
        post_game_team_elo.append(R_A_NEW)
        post_game_opponent_elo.append(R_B_NEW)

    games_df["Pre-Game Team Elo"] = pre_game_team_elo
    games_df["Pre-Game Opponent Elo"] = pre_game_opponent_elo
    games_df["Post-Game Team Elo"] = post_game_team_elo
    games_df["Post-Game Opponent Elo"] = post_game_opponent_elo
    # print(sorted(team_elos.items(), key=lambda x: x[1], reverse=True))

    games_df.to_csv(f'game-data/{year}games.csv')
    print(f'{j + 1}/{len(years)} | added elo for games from {year}')


1/18 | added elo for games from 2008
2/18 | added elo for games from 2009
3/18 | added elo for games from 2010
4/18 | added elo for games from 2011
5/18 | added elo for games from 2012
6/18 | added elo for games from 2013
7/18 | added elo for games from 2014
8/18 | added elo for games from 2015
9/18 | added elo for games from 2016
10/18 | added elo for games from 2017
11/18 | added elo for games from 2018
12/18 | added elo for games from 2019
13/18 | added elo for games from 2020
14/18 | added elo for games from 2021
15/18 | added elo for games from 2022
16/18 | added elo for games from 2023
17/18 | added elo for games from 2024
18/18 | added elo for games from 2025


### Accuracy of Pure Elo Prediction for Tournament (from round of 64):

Heuristic: Team with higher Elo is more likely to win (pretty straight forward)

32 + 16 + 8 + 4 + 2 + 1 = **63 games** for each March Madness (starting from 64 teams). Thus, the games relevant to March Madness will be the latest **126 entries** (games are double counted for each team).

Update: there's fucking random games (non-tournament games) that occur after the tournament start date, so I'll just have to manually set each March Madness start date...

In [150]:
round_of_64_start_dates = {
    2008: "2008-03-20",
    2009: "2009-03-19",
    2010: "2010-03-18",
    2011: "2011-03-17",
    2012: "2012-03-15",
    2013: "2013-03-21",
    2014: "2014-03-20",
    2015: "2015-03-19",
    2016: "2016-03-17",
    2017: "2017-03-16",
    2018: "2018-03-15",
    2019: "2019-03-21",
    2020: "2020-03-19",
    2021: "2021-03-18",
    2022: "2022-03-17",
    2023: "2023-03-16",
    2024: "2024-03-21",
    2025: "2025-03-20"
}

In [151]:
import random

print('Accuracy of Pure Elo Prediction for Tournament (from round of 64):')
for j, year in enumerate(years[:-1]): # want to exclude 2025, not full season
    games_df = pd.read_csv(f'game-data/{year}games.csv', index_col='Unnamed: 0')
    round_of_64_start_date = round_of_64_start_dates[year]
    tournament_games_df = games_df.loc[games_df['Date'] >= round_of_64_start_date]

    seen_games = set() # df['Game Unique ID']
    errors = 0

    for i, row in tournament_games_df.iterrows():
        team_A, team_B = row['Team'], row['Opponent']
        team_A_elo, team_B_elo = row['Pre-Game Team Elo'], row['Pre-Game Opponent Elo']
        game_unique_id, result = row['Game Unique ID'], row['Result']

        if game_unique_id in seen_games:
            continue  # avoid duplicate games

        if (team_A_elo > team_B_elo and result == 'L') or (team_A_elo < team_B_elo and result == 'W'):
            errors += 1
        elif team_A_elo == team_B_elo: 
            if result == "L" and random.random() < 0.5:
                errors += 1

        seen_games.add(game_unique_id)

    total_games = len(seen_games)
    sample_error_rate = errors / total_games if total_games > 0 else 0 
    print(f"{year} - Sample Error Rate: {sample_error_rate:.4f}")



Accuracy of Pure Elo Prediction for Tournament (from round of 64):
2008 - Sample Error Rate: 0.3218
2009 - Sample Error Rate: 0.3579
2010 - Sample Error Rate: 0.3298
2011 - Sample Error Rate: 0.3673
2012 - Sample Error Rate: 0.4466
2013 - Sample Error Rate: 0.4175
2014 - Sample Error Rate: 0.4314
2015 - Sample Error Rate: 0.3431
2016 - Sample Error Rate: 0.3491
2017 - Sample Error Rate: 0.3429
2018 - Sample Error Rate: 0.3636
2019 - Sample Error Rate: 0.3810
2020 - Sample Error Rate: 0.0000
2021 - Sample Error Rate: 0.4368
2022 - Sample Error Rate: 0.4038
2023 - Sample Error Rate: 0.3226
2024 - Sample Error Rate: 0.4105


In [152]:
games_df = pd.read_csv(f'game-data/2020games.csv', index_col='Unnamed: 0')
games_df.tail(10)

,Date,Team,Conference,Opponent,Venue,Result,AdjO,AdjD,EffO,eFG%,...,Opponent Coach,Unknown,Game Importance,Team Points,Opponent Points,Point Differential,Pre-Game Team Elo,Pre-Game Opponent Elo,Post-Game Team Elo,Post-Game Opponent Elo
7497,2020-03-11,Saint Joseph's,A10,George Mason,N,L,99.3,111.6,99.3,49.2,...,Dave Paulsen,-5.88750,0.493610,70,77,-7,1307,1474,1297,1484
7496,2020-03-11,George Mason,A10,Saint Joseph's,N,W,103.2,98.5,109.3,50.8,...,Billy Lange,5.88750,0.312063,77,70,7,1474,1307,1484,1297
3565,2020-03-11,Northern Arizona,BSky,Idaho St.,N,L,82.5,96.3,91.2,39.1,...,Ryan Looney,-2.39225,0.227875,62,64,-2,1450,1285,1433,1302
7278,2020-03-11,FIU,CUSA,Rice,N,W,111.9,104.7,119.4,60.3,...,Scott Pera,3.04231,0.448603,85,76,9,1496,1469,1516,1449
3564,2020-03-11,Idaho St.,BSky,Northern Arizona,N,W,90.0,90.9,94.1,50.0,...,Shane Burcar,2.39225,0.337460,64,62,2,1285,1450,1302,1433
6204,2020-03-11,North Carolina,ACC,Syracuse,N,L,80.2,101.0,74.9,35.0,...,Jim Boeheim,-13.29480,0.834526,53,81,-28,1582,1586,1549,1619
7279,2020-03-11,Rice,CUSA,FIU,N,L,109.3,116.1,106.7,54.4,...,Jeremy Ballard,-3.04231,0.568226,76,85,-9,1469,1496,1449,1516
9860,2020-03-11,Arkansas,SEC,Vanderbilt,N,W,119.3,96.4,118.6,60.8,...,Jerry Stackhouse,11.22320,0.532926,86,73,13,1595,1470,1611,1454
321,2020-03-11,Texas St.,SB,Appalachian St.,H,W,116.8,98.3,117.1,63.2,...,Dustin Kerns,11.04590,0.338186,85,68,17,1618,1506,1636,1488
2765,2020-03-11,George Washington,A10,Fordham,N,L,85.9,121.3,77.7,43.3,...,Jeff Neubauer,-9.36769,0.328867,52,72,-20,1410,1352,1374,1388


### Full Season Accuracy using Elo Scores

I have a feeling that the first games of the season will tank error rates (all 50/50)

In [153]:
import random

print('Full Season Accuracy using Elo Scores')
for j, year in enumerate(years):
    games_df = pd.read_csv(f'game-data/{year}games.csv', index_col='Unnamed: 0')

    seen_games = set() # df['Game Unique ID']
    errors = 0

    for i, row in games_df.iterrows():
        team_A, team_B = row['Team'], row['Opponent']
        team_A_elo, team_B_elo = row['Pre-Game Team Elo'], row['Pre-Game Opponent Elo']
        game_unique_id, result = row['Game Unique ID'], row['Result']

        if game_unique_id in seen_games:
            continue  # avoid duplicate games

        if (team_A_elo > team_B_elo and result == 'L') or (team_A_elo < team_B_elo and result == 'W'):
            errors += 1
        elif team_A_elo == team_B_elo: 
            if result == "L" and random.random() < 0.5:
                errors += 1

        seen_games.add(game_unique_id)

    total_games = len(seen_games)
    sample_error_rate = errors / total_games if total_games > 0 else 0 
    print(f"{year} - Sample Error Rate: {sample_error_rate:.4f}")



Full Season Accuracy using Elo Scores
2008 - Sample Error Rate: 0.3092
2009 - Sample Error Rate: 0.3154
2010 - Sample Error Rate: 0.2997
2011 - Sample Error Rate: 0.3058
2012 - Sample Error Rate: 0.2920
2013 - Sample Error Rate: 0.3126
2014 - Sample Error Rate: 0.3106
2015 - Sample Error Rate: 0.3104
2016 - Sample Error Rate: 0.3062
2017 - Sample Error Rate: 0.3086
2018 - Sample Error Rate: 0.3105
2019 - Sample Error Rate: 0.3189
2020 - Sample Error Rate: 0.3215
2021 - Sample Error Rate: 0.3329
2022 - Sample Error Rate: 0.2964
2023 - Sample Error Rate: 0.3250
2024 - Sample Error Rate: 0.3292
2025 - Sample Error Rate: 0.3081
